# Activity 1: Monte Carlo Exploration

**Duration**: 2 hours

**Learning Objectives**:
- Understand how Monte Carlo simulation captures particle transport physics
- Identify characteristic features of proton energy deposition (Bragg peak)
- Investigate the relationship between incident energy and peak position

## Background

Monte Carlo simulation tracks individual particles through matter, sampling from probability distributions for each interaction. OpenTOPAS (built on Geant4) simulates the full physics of proton transport including:
- Electromagnetic interactions (ionization, multiple scattering)
- Nuclear interactions (at higher energies)
- Energy straggling (statistical fluctuations)

In this activity, you will explore pre-computed simulation outputs to understand the physics revealed by this computational technique.

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('..')

# Set style for publication-quality figures
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## Part 1: Loading Simulation Data

The TOPAS simulations were run at 14 different energies, with 10,000 proton histories each. Let's load and examine the data.

In [ ]:
# Load the training data
data_path = '../data/pinn_training_data_v2.csv'  # Symlink to pinn_data
# Alternative path if symlink doesn't work:
# data_path = '../../pinn_data/pinn_training_data_v2.csv'

try:
    df = pd.read_csv(data_path)
except FileNotFoundError:
    df = pd.read_csv('../../pinn_data/pinn_training_data_v2.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nEnergies simulated: {sorted(df['energy_MeV'].unique())} MeV")
df.head()

### 📝 Question 1
How many unique energies were simulated? What is the energy range?

*Your answer:*

## Part 2: Dose-Depth Profiles

The characteristic feature of proton energy deposition is the **Bragg peak** - a sharp increase in energy deposition near the end of the particle's range.

In [ ]:
# Plot dose-depth curves for different energies
fig, ax = plt.subplots(figsize=(12, 8))

# Select a subset of energies to visualize
energies_to_plot = [70, 100, 150, 200, 300, 500, 1000, 2000]
colors = plt.cm.viridis(np.linspace(0, 1, len(energies_to_plot)))

for energy, color in zip(energies_to_plot, colors):
    subset = df[df['energy_MeV'] == energy]
    if len(subset) > 0:
        ax.plot(subset['depth_mm'], subset['dEdx_MeV_per_mm'], 
                label=f'{energy} MeV', color=color, linewidth=2)

ax.set_xlabel('Depth in EJ-200 (mm)', fontsize=14)
ax.set_ylabel('Energy Deposition dE/dx (MeV/mm)', fontsize=14)
ax.set_title('Proton Energy Deposition Profiles in EJ-200 Scintillator', fontsize=16)
ax.legend(title='Incident Energy', loc='upper right')
ax.set_xlim(0, 10)
plt.tight_layout()
plt.show()

### 📝 Question 2
For which energies do you observe a clear Bragg peak within the 10mm detector? For which energies does the peak occur outside the detector?

*Your answer:*

## Part 3: Bragg Peak Analysis

Let's quantitatively analyze the Bragg peak position as a function of energy.

In [ ]:
# Find peak position and height for each energy
peak_data = []

for energy in sorted(df['energy_MeV'].unique()):
    subset = df[df['energy_MeV'] == energy]
    if len(subset) > 0:
        idx_max = subset['dEdx_MeV_per_mm'].idxmax()
        peak_depth = subset.loc[idx_max, 'depth_mm']
        peak_dose = subset.loc[idx_max, 'dEdx_MeV_per_mm']
        entrance_dose = subset[subset['depth_mm'] < 0.5]['dEdx_MeV_per_mm'].mean()
        peak_data.append({
            'energy_MeV': energy,
            'peak_depth_mm': peak_depth,
            'peak_dEdx': peak_dose,
            'entrance_dEdx': entrance_dose,
            'peak_to_entrance_ratio': peak_dose / entrance_dose if entrance_dose > 0 else np.nan
        })

peak_df = pd.DataFrame(peak_data)
print(peak_df.to_string(index=False))

In [ ]:
# Plot peak position vs energy
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Peak depth vs energy
ax1 = axes[0]
low_energy = peak_df[peak_df['energy_MeV'] <= 300]
ax1.scatter(low_energy['energy_MeV'], low_energy['peak_depth_mm'], s=100, c='blue')
ax1.set_xlabel('Incident Energy (MeV)', fontsize=14)
ax1.set_ylabel('Peak Depth (mm)', fontsize=14)
ax1.set_title('Bragg Peak Position vs Energy', fontsize=14)
ax1.axhline(y=10, color='red', linestyle='--', label='Detector limit')
ax1.legend()

# Right: Peak-to-entrance ratio
ax2 = axes[1]
ax2.scatter(peak_df['energy_MeV'], peak_df['peak_to_entrance_ratio'], s=100, c='green')
ax2.set_xlabel('Incident Energy (MeV)', fontsize=14)
ax2.set_ylabel('Peak / Entrance Dose Ratio', fontsize=14)
ax2.set_title('Bragg Peak Enhancement', fontsize=14)
ax2.set_xscale('log')

plt.tight_layout()
plt.show()

### 📝 Question 3
Based on your observations:
1. How does peak position change with energy? Is the relationship linear?
2. Why does the peak-to-entrance ratio decrease at higher energies?
3. At approximately what energy does the proton range exceed the 10mm detector?

*Your answers:*

## Part 4: The Bethe-Bloch Formula

The mean energy loss per unit path length is described by the Bethe-Bloch formula:

$$-\left\langle\frac{dE}{dx}\right\rangle = \frac{4\pi n_e z^2 e^4}{m_e c^2 \beta^2} \left[\ln\left(\frac{2m_e c^2 \beta^2 \gamma^2}{I}\right) - \beta^2\right]$$

where $\beta = v/c$ and $\gamma = 1/\sqrt{1-\beta^2}$.

Notice the $\beta^{-2}$ dependence: as particles slow down ($\beta \to 0$), they lose energy faster!

In [ ]:
# Extract mean stopping power (entrance region) vs relativistic parameters
entrance_data = df[df['depth_mm'] < 1.0].groupby('energy_MeV').agg({
    'dEdx_MeV_per_mm': 'mean',
    'beta': 'first',
    'gamma': 'first'
}).reset_index()

fig, ax = plt.subplots(figsize=(10, 6))

# Plot dE/dx vs beta^{-2}
beta_inv_sq = 1 / (entrance_data['beta']**2)
ax.scatter(beta_inv_sq, entrance_data['dEdx_MeV_per_mm'], s=100, c='blue', alpha=0.7)

ax.set_xlabel(r'$\beta^{-2}$ (relativistic parameter)', fontsize=14)
ax.set_ylabel('Mean dE/dx at entrance (MeV/mm)', fontsize=14)
ax.set_title('Stopping Power vs Relativistic Velocity', fontsize=16)

# Add energy labels
for _, row in entrance_data.iterrows():
    if row['energy_MeV'] in [70, 200, 1000, 6000]:
        ax.annotate(f"{int(row['energy_MeV'])} MeV", 
                    (1/row['beta']**2, row['dEdx_MeV_per_mm']),
                    textcoords="offset points", xytext=(10,5), fontsize=10)

plt.tight_layout()
plt.show()

### 📝 Question 4
Does the simulation data show the expected $\beta^{-2}$ dependence from Bethe-Bloch theory? Where does the relationship break down and why?

*Your answer:*

## Part 5: Extension Activity

**Challenge**: Predict what would happen if we used a thicker detector (20mm instead of 10mm).

1. For which energies would we now see Bragg peaks?
2. How would the peak-to-entrance ratio change?

Sketch your predicted dose profiles below, then discuss with your instructor.

---

## Summary

In this activity, you learned:

1. **Monte Carlo simulations produce dose-depth profiles** that reveal the physics of particle transport
2. **The Bragg peak** is a characteristic feature where protons deposit maximum energy near end-of-range
3. **Peak position increases with energy** because higher-energy protons travel further before stopping
4. **The Bethe-Bloch formula** explains the $\beta^{-2}$ dependence of stopping power

### Key Teaching Point

Monte Carlo simulation provides "ground truth" for validation—**but only within the physics models selected**. The choice of physics list (electromagnetic processes, nuclear interactions) involves assumptions with physical consequences.